In [ ]:
import pandas as pd
import os
from tqdm import tqdm
from openai import OpenAI

# --- 1. 설정 ---
API_KEY = ""
INPUT_CSV = "hard_samples_for_llm.csv"
OUTPUT_CSV = "gold_labels.csv"    # 결과 저장할 파일
LIMIT = 950  # [안전장치]

# API 키 확인
if API_KEY == "YOUR_OPENAI_API_KEY":
    raise ValueError("❌ API Key를 설정해주세요!")

client = OpenAI(api_key=API_KEY)

# --- 2. LLM 호출 함수 ---
def get_gold_labels(text, candidates):
    # 프롬프트: 헷갈리는 리뷰를 주고, 후보군 중에서 2~3개를 고르라고 시킴
    prompt = f"""
    You are an expert classifier for Amazon products.
    
    Task: Identify the correct product categories for the review below.
    Constraint: Select exactly 2 or 3 most relevant category IDs from the Candidate List.
    
    Review: "{text}"
    
    Candidate List (ID: Name):
    {candidates}
    
    Output Format:
    Return ONLY the IDs, separated by commas. (e.g., 12, 45, 108)
    Do not write any explanation.
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # 가성비 좋은 모델 사용
            messages=[{"role": "user", "content": prompt}],
            temperature=0.0       # 일관된 답변을 위해 0으로 설정
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"⚠️ API Error: {e}")
        return None

# --- 3. 메인 실행 ---
if not os.path.exists(INPUT_CSV):
    print(f"❌ Error: {INPUT_CSV} 파일이 없습니다. 'find_hard_samples.py'를 먼저 실행하세요.")
else:
    df = pd.read_csv(INPUT_CSV)
    print(f"Total hard samples available: {len(df)}")
    
    # 안전하게 50개만 자르기
    target_df = df.head(LIMIT)
    results = []

    print(f"🚀 Starting Pilot Test with {LIMIT} samples...")

    for idx, row in tqdm(target_df.iterrows(), total=len(target_df)):
        pid = row['pid']
        text = row['text']
        # 후보군 포맷팅 (보기 좋게 줄바꿈)
        candidates = row['candidates'].replace(" | ", "\n")
        
        # LLM에게 질문
        labels = get_gold_labels(text, candidates)
        
        # 응답이 있으면 저장
        if labels:
            # 혹시 모를 공백이나 마침표 제거 후 저장
            clean_labels = labels.replace(".", "").strip()
            results.append([pid, clean_labels])

    # --- 4. 결과 저장 ---
    if results:
        final_df = pd.DataFrame(results, columns=['id', 'labels'])
        final_df.to_csv(OUTPUT_CSV, index=False)
        
    else:
        print("❌ No labels generated. Check your API Key or Network.")